# Stacking is not necessarily optimal, but that's OK because projections can't really be optimized

This notebook shows how stacking is not necessarily optimal when projections are correct (or when looking at 'perfect' lineups). However, stacking may make sense as a heuristic for dealing with the substantial inaccuracies that plague even the best projections.

In [1]:
import itertools
from pprint import pprint
import pandas as pd
from optimize import optimize_lineup

In [2]:
season = pd.read_csv('data/results/results_2020.csv')
results = []

In [24]:
from nflschedule import main_slate_teams

ModuleNotFoundError: No module named 'nflschedule'

In [19]:
fn = 'data/optimals/optimal_2020.pkl'

try:
    comb = pd.read_pickle(fn)

except FileNotFoundError:
    mapping = {
        'noconstraints': (None, None),
        'nostack': (0, None),
        'stack1': (1, None),
        'bb1': (None, 1),
        'stack1bb1': (1, 1)
    }

    for week in range(1, 17):
        print(f'Starting week {week}')

        # can speed up search by limiting pool to feasible candidates
        wdf = season.loc[(season.week == week) & (season.fppg > 5), :]

        # loop through each strategy
        dfs = []
        for cat, params in mapping.items():
            dfs.append(optimize_lineup(wdf, stack=params[0], bringback=params[1]).assign(strategy=cat))
        results.append(dfs)

    # results is a list of list, so use itertools to concat
    comb = pd.concat(itertools.chain.from_iterable(results))

    # save time by pickling results
    comb.to_pickle(fn)

In [23]:
(
  comb
  .groupby(['strategy', 'week'], as_index=False)
  .agg(score=('fppg', 'sum'))
  .pivot(index='strategy', columns='week', values='score')
  .assign(avg=lambda df_: df_.mean(axis=1))
  .sort_values('avg', ascending=False)
)

week,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,avg
strategy,,,,,,,,,,,,,,,,,
noconstraints,288.78,293.50,314.5,316.98,289.2,294.00,321.42,281.84,291.40,247.54,257.82,312.82,301.14,268.22,269.92,325.74,292.17625
nostack,288.78,293.50,314.5,311.16,286.1,290.24,321.42,275.84,291.40,247.54,257.82,310.08,300.66,268.22,269.92,325.74,290.80750
stack1,287.76,288.68,311.3,316.98,289.2,294.00,318.58,281.84,284.82,247.34,254.50,312.82,301.14,264.24,264.74,323.80,290.10875
bb1,288.78,293.50,310.7,316.98,283.5,293.64,319.74,271.84,285.12,243.32,257.82,308.42,300.66,262.22,269.72,316.04,288.87500
stack1bb1,287.76,287.00,310.7,316.98,281.1,293.64,311.44,271.84,282.28,242.70,252.76,308.42,294.16,259.94,263.92,306.88,285.72000


In most weeks, the 'nostack' strategy (a rule prohibiting stacking) works as well as a rule requiring stacking (either 'stack1' or 'bb1'). A rule requiring a stack and a bringback consistently produces worse optimals, although the overall difference is roughly seven points on average.

So why do experts recommend stacking and bringbacks if they don't result in optimal lineups? There are two (related) reasons. 

Second, you don't need the true optimal lineup to win a tournament. For smaller tournaments, you only need to beat your opponents. Even in very large tournaments (for 10+ game slates), you don't need the optimal lineup to win. The table below shows the score of the main-slate MillyMaker winner and the score of the optimal lineup.

In [65]:
comb.loc[comb.week == 10, :].groupby('strategy').agg(sal=('salary', 'sum'), pts=('fppg', 'sum'))

,sal,pts
strategy,,
bb1,50000,243.32
noconstraints,45300,219.64
nostack,45300,219.64
stack1,45200,219.44
stack1bb1,45400,214.80


In [68]:
comb.loc[comb.week == 10, 'full_name'].value_counts()

Alvin_Kamara          5
Las_Vegas_Defense     5
Cole_Beasley          5
DeAndre_Hopkins       5
Josh_Jacobs           5
Tom_Brady             3
Hunter_Henry          3
Ronald_Jones          3
Nyheim_Hines          2
Kyler_Murray          1
Tee_Higgins           1
Cameron_Brate         1
Ben_Roethlisberger    1
Rob_Gronkowski        1
Name: full_name, dtype: int64

In [107]:
season.loc[(season.week == 10) & (season.full_name.str.contains('Marq')), :]

,year,week,full_name,pos,team,opp,salary,fppg
3766,2020,10,Marquez_Valdes-Scantling,WR,GNB,JAC,4400,27.9
3843,2020,10,Marquise_Brown,WR,BAL,NWE,5700,3.4
3882,2020,10,Marquez_Callaway,WR,NOR,SFO,3000,0.0


# Projections are quite inaccurate, so stacking may help compensate for inevitable errors